In [1]:
import torch
from torch import nn
from models import resnet
path = '../../../data/bin/resnet_10.pth'

In [3]:
model = resnet.myresnet(
    sample_input_W=193,
    sample_input_H=229,
    sample_input_D=193,
    shortcut_type='B',
    no_cuda=False
)
net_dict = model.state_dict()
pretrain = torch.load(path, map_location='cuda:0')
# remove 'module.'
pretrain_dict = {k[7:]: v for k, v in pretrain['state_dict'].items() if k[7:] in net_dict.keys()}

net_dict.update(pretrain_dict)
model.load_state_dict(net_dict)

<All keys matched successfully>

In [4]:
import sys
import os
import nibabel as nib
import numpy as np
import pandas as pd
sys.path.append('../..')
from src.utils.data import writePandas, getPandas, getConfig, getDict
os.chdir('../../..')

In [5]:
data = getPandas('pat_data')
conf = getConfig('data')
train_idx = conf['indices']['pat']['train']
test_idx = conf['indices']['pat']['test']
keys = data['KEY'].values
paths = data['ANTs_Reg'].values

In [5]:
rst = np.array([])
model.eval()
for path in paths:
    print(path)
    img = nib.load(path).get_fdata()
    i_min = img.min()
    i_std = img.std()
    img = (img - i_min) / i_std
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=0)
    img = torch.from_numpy(img)
    img = img.cpu().float()
    output = model(img)
    output = output.cpu().detach().numpy()
    rst = np.append(rst, output)

data/subj/3107V04I296431/ants/reg.nii.gz
data/subj/3107V06I378218/ants/reg.nii.gz
data/subj/3107V10I498876/ants/reg.nii.gz
data/subj/3108V06I378222/ants/reg.nii.gz


In [6]:
model = model.cuda()
model.eval()
img = nib.load('data/subj/3107V04I296431/ants/reg.nii.gz').get_fdata()
i_min = img.min()
i_std = img.std()
img = (img - i_min) / i_std
img = np.expand_dims(img, axis=0)
img = np.expand_dims(img, axis=0)
img = torch.from_numpy(img)
#img = img.cpu().float()
img = img.cuda().float()
output = model(img)
output = output.cpu().detach().numpy()
print(output.shape)
print(np.count_nonzero(output))

(1, 76800)
38856
